## General notes
Use this notebook if you want to generate a .csv for a pre-trained model starting from a certain epoch

**N.B.: It is useful when applying early stopping**

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import os
import tensorflow as tf
import numpy as np
 
SEED = 1234
tf.random.set_seed(SEED)  

## We build the model exactly as it was when trained
We distinguish the cases in which:
    - we use a pretrained network
    - we use a network built from scratch
In both cases it is required to copy the **exact** structure of the model

In [6]:
#img size
img_h = 224
img_w = 224

#batches
bs = 10

num_classes = 3

transfer_learning = True

model = tf.keras.Sequential()

if transfer_learning:
    cnn = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
else:
    starting_feat = 32
    depth = 5

    # Features extraction
    for i in range(depth):
        if i == 0:
            input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]

        # Conv block: Conv2D -> Activation -> Pooling
        if i == 0 or i == 1:
            repeat = 2
        else:
            repeat = 3
        for reps in range(repeat):
            model.add(tf.keras.layers.Conv2D(filters=starting_feat, 
                                             kernel_size=(3, 3),
                                             strides=(1, 1),
                                             padding='same',
                                             input_shape=input_shape))
            model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

# Classifier
model.add(cnn)
for i in range(4):
    model.add(tf.keras.layers.Conv2D(filters=512, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same'))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

#Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 512)         2359808   
_________________________________________________________________
re_lu (ReLU)                 (None, 7, 7, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 512)         2048      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 512)         2359808   
_________________________________________________________________
re_lu_1 (ReLU)               (None, 7, 7, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 512)        

## Let's load the weights

In [8]:
import os

def get_model_path(model_name, epoch_number):
    path = os.path.join(os.getcwd(), 'tensorboard_experiments', model_name, 'ckpts')
    for file in os.listdir(path):
        if str(epoch_number) in file:
            if epoch_number in range(10):
                path = os.path.join(path, "cp_0" + str(epoch_number) + ".ckpt")
            else:
                path = os.path.join(path, "cp_" + str(epoch_number) + ".ckpt")
            return path

In [30]:
model_name = np.array(os.listdir(os.path.join(os.getcwd(), 'tensorboard_experiments')))[-1]
epochs = 33                              # <--- modify this

path = get_model_path(model_name, epochs)

model.load_weights(path)

### - optimization parameters

In [31]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy() # because we have one-hot encoded labels (as specified by the 
                                                 # class_mode parameter in flow_from_dataframe())

# Learning rate
lr = (1e-4)   # <-- modify this

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

### - test generator

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'MaskDataset')
test_dir = os.path.join(dataset_dir, 'test')
test_data_gen = ImageDataGenerator(rescale=1./255)
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(img_h, img_w), 
                                             color_mode='rgb',
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 450 images belonging to 1 classes.


## Then, let's generate a .csv corresponding to the model with the loaded weights

In [33]:
import ntpath

predictions = model.predict(test_gen, verbose=1)

results = {}

images = test_gen.filenames
i = 0

for p in predictions:
    prediction = np.argmax(p)
    image_name = ntpath.basename(images[i])
    results[image_name] = str(prediction)
    i += 1

45/45 [==============================] - 19s 411ms/step


In [34]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [35]:
exps_dir = os.path.join('tensorboard_experiments')
exp_dir = os.path.join(exps_dir, model_name)
create_csv(results, exp_dir)